# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [36]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv(os.path.join("..","output_data", "cities.csv"))
cities_df.head()

,Unnamed: 0,City,Lat,Lng,Temp (C),Humidity,Cloudiness,Wind_Speed (kmph),Date,Country
0,0,Muli,2.92,73.57,27.91,76,100,14.868,1601610195,MV
1,1,Busselton,-33.65,115.33,17.22,43,36,26.388,1601610195,AU
2,2,Bindi,9.92,-11.45,21.54,97,100,5.220,1601610196,SL
3,3,Kapaa,22.08,-159.32,29.00,66,1,14.760,1601610196,US
4,4,Varaždin,46.30,16.34,8.89,99,89,6.336,1601610197,HR


In [3]:
len(cities_df)

600

In [4]:
cities_df.dtypes

Unnamed: 0             int64
City                  object
Lat                  float64
Lng                  float64
Temp (C)             float64
Humidity               int64
Cloudiness             int64
Wind_Speed (kmph)    float64
Date                   int64
Country               object
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [6]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]]
locations.dtypes

Lat    float64
Lng    float64
dtype: object

In [7]:
# Store 'Humidity' as weight 
weight = cities_df["Humidity"].astype(float)
weight

0      76.0
1      43.0
2      97.0
3      66.0
4      99.0
       ... 
595    70.0
596    69.0
597    84.0
598    73.0
599    65.0
Name: Humidity, Length: 600, dtype: float64

In [8]:
# Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Ideal conditions: 

- A max temperature (celcius) lower than 31 degrees but higher than 24 degrees celcius.

- Wind speed less than 25 kmph.

- Zero cloudiness.

- Humidity below 60

In [9]:
#converting temperature from Kelvin to Celcius 
ideal_conditions = cities_df.loc[(cities_df["Temp (C)"]> 24) & (cities_df["Temp (C)"]<31)][cities_df["Wind_Speed (kmph)"] < 25][cities_df["Cloudiness"] == 0][cities_df['Humidity']<60]
ideal_conditions

C:\Users\ezgal\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,Unnamed: 0,City,Lat,Lng,Temp (C),Humidity,Cloudiness,Wind_Speed (kmph),Date,Country
58,58,Guerrero Negro,27.98,-114.06,26.07,52,0,16.956,1601610237,MX
62,62,Waddān,29.16,16.14,25.07,37,0,14.292,1601610240,LY
168,168,Rāmhormoz,31.28,49.60,29.68,31,0,2.160,1601610348,IR
228,228,San Quintín,30.48,-115.95,25.24,46,0,11.088,1601610422,MX
275,275,Mayor Pablo Lagerenza,-19.93,-60.77,29.73,28,0,12.132,1601610488,PY
303,303,Ulladulla,-35.35,150.47,27.00,33,0,9.360,1601610527,AU
306,306,Lakes Entrance,-37.88,147.98,27.78,33,0,11.268,1601610530,AU
381,381,Flinders,-34.58,150.86,25.56,43,0,6.444,1601610646,AU
389,389,Sabha,27.04,14.43,24.89,32,0,9.396,1601610661,LY
393,393,Alīganj,27.50,79.18,29.51,41,0,2.844,1601610666,IN


In [14]:
len(ideal_conditions)

6

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = ideal_conditions
hotel_df['Hotel Name'] = ""
hotel_df['Rating'] = ''
hotel_df = hotel_df.drop(["Unnamed: 0","Date", "Cloudiness"], axis = 1).reset_index().drop("index", axis = 1)

In [11]:
hotel_df

,City,Lat,Lng,Temp (C),Humidity,Wind_Speed (kmph),Country,Hotel Name,Rating
0,Guerrero Negro,27.98,-114.06,26.07,52,16.956,MX,,
1,Waddān,29.16,16.14,25.07,37,14.292,LY,,
2,Rāmhormoz,31.28,49.60,29.68,31,2.160,IR,,
3,San Quintín,30.48,-115.95,25.24,46,11.088,MX,,
4,Mayor Pablo Lagerenza,-19.93,-60.77,29.73,28,12.132,PY,,
5,Ulladulla,-35.35,150.47,27.00,33,9.360,AU,,
6,Lakes Entrance,-37.88,147.98,27.78,33,11.268,AU,,
7,Flinders,-34.58,150.86,25.56,43,6.444,AU,,
8,Sabha,27.04,14.43,24.89,32,9.396,LY,,
9,Alīganj,27.50,79.18,29.51,41,2.844,IN,,


In [121]:
hotel_df['City'].iloc[1]

'San Quintín'

In [122]:
hotel_df.dtypes

City                  object
Lat                  float64
Lng                  float64
Temp (C)             float64
Humidity               int64
Wind_Speed (kmph)    float64
Hotel Name            object
dtype: object

In [12]:
lats = hotel_df['Lat'].astype(str) 
longs = hotel_df['Lng'].astype(str)

#initialising index for data frame
index = 0
for lat, lng in zip(lats, longs):
    
    # set up a parameters dictionary
    params = {
        "location": f"{lat}, {lng}",
        "radius": 5000,
        "type": "hotel",
        "keyword": "hotel",
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_name = response['results'][0]['name']
        hotel_df['Hotel Name'].iloc[index] = hotel_name
    
    except:
        city_name = hotel_df['City'].iloc[index]
        print(f"Sorry, not hotel found for {city_name}")
           
    try:
        rating = response['results'][0]['rating']
        hotel_df['Rating'].iloc[index] = rating
    
    except:
          print(f"Sorry, no rating found for {hotel_name} in {city_name}")
            
    index +=1 
        
       

C:\Users\ezgal\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Sorry, not hotel found for Waddān
Sorry, no rating found for Casa Laguna, Bed & Breakfast in Waddān
Sorry, not hotel found for Mayor Pablo Lagerenza
Sorry, no rating found for Old Mill Hotel in Mayor Pablo Lagerenza
Sorry, not hotel found for Alīganj
Sorry, no rating found for فندق المواجيد in Alīganj
Sorry, not hotel found for Moussoro
Sorry, no rating found for Pippi's At The Point Hotel in Moussoro
Sorry, not hotel found for Marzuq
Sorry, no rating found for Pippi's At The Point Hotel in Marzuq
Sorry, not hotel found for Taoudenni
Sorry, no rating found for Pippi's At The Point Hotel in Taoudenni


In [13]:
hotel_df

,City,Lat,Lng,Temp (C),Humidity,Wind_Speed (kmph),Country,Hotel Name,Rating
0,Guerrero Negro,27.98,-114.06,26.07,52,16.956,MX,"Casa Laguna, Bed & Breakfast",4.7
1,Waddān,29.16,16.14,25.07,37,14.292,LY,,
2,Rāmhormoz,31.28,49.60,29.68,31,2.160,IR,مجتمع رفاهی شهید رجایی,3.5
3,San Quintín,30.48,-115.95,25.24,46,11.088,MX,Old Mill Hotel,4.5
4,Mayor Pablo Lagerenza,-19.93,-60.77,29.73,28,12.132,PY,,
5,Ulladulla,-35.35,150.47,27.00,33,9.360,AU,Bannisters Pavilion Rooftop Bar & Grill,4.3
6,Lakes Entrance,-37.88,147.98,27.78,33,11.268,AU,The Esplanade Resort & Spa,4.1
7,Flinders,-34.58,150.86,25.56,43,6.444,AU,Lakeview Hotel Motel,3.8
8,Sabha,27.04,14.43,24.89,32,9.396,LY,فندق المواجيد,0
9,Alīganj,27.50,79.18,29.51,41,2.844,IN,,


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""


# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
# adding the info boxes to the map
symbol_layer = gmaps.symbol_layer(locations, info_box_content=hotel_info)
fig.add_layer(symbol_layer)

In [16]:
# adding the markers and displaying final map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))